In [1]:
# from google.colab import drive
# drive.mount('/content/drive',force_remount=True)

In [2]:
# !pip install import-ipynb

In [3]:
import import_ipynb

In [4]:
# %cd /content/drive/MyDrive/A62_projet_synthese_feature/feature-colab-from-git/notebook

In [5]:
%run iteration_0_imports.py

In [6]:
%run iteration_0_parameters.py

In [7]:
import iteration_0_utils as utils

importing Jupyter notebook from iteration_0_utils.ipynb


In [8]:
%whos

Variable                          Type        Data/Info
-------------------------------------------------------
HEALTHY                           str         H
IMG_IN_COLOR                      int         1
IMG_SIZE                          int         64
Image                             module      <module 'PIL.Image' from <...>packages\\PIL\\Image.py'>
NBR_SAMPLE                        int         20000
PATH_DATA                         str         ../data
PATH_DATA_EDA                     str         ../data/eda
PATH_DATA_LIST                    str         ../data/list
PATH_DATA_SAMPLE                  str         ../data/sample
PATH_MODEL                        str         ../model
PATH_ROOT                         str         ..
PATH_SOURCE_IMG                   str         ../cell_images
RandomForestClassifier            ABCMeta     <class 'sklearn.ensemble.<...>.RandomForestClassifier'>
SICK                              str         S
classification_report             function

In [9]:
df_path_img_sick = pd.read_csv(PATH_DATA_LIST + "/" + "df_list_path_sick_img.csv")
df_path_img_healthy = pd.read_csv(PATH_DATA_LIST + "/" + "df_list_path_healthy_img.csv")
df_path_img = pd.read_csv(PATH_DATA_LIST + "/" + "df_list_path_img.csv")

# Prétraitement

## positif_global_std

In [10]:
def image_resize(path_img: str) -> np.array:
    img_arr = cv2.imread(path_img, flags = IMG_IN_COLOR)
    
    h, w, _ = img_arr.shape
    ratio = IMG_SIZE / max(h,w) 
    img_arr_resize = cv2.resize(img_arr, dsize = (int(np.ceil(w*ratio)), int(np.ceil(h*ratio)) ))
    return img_arr_resize

In [11]:
def positive_global_std(img_arr: np.array) -> np.array:
    
    mean = img_arr.mean()
    std  = img_arr.std()
    img_arr_std = (img_arr - mean) / std
    
    # clip les valeurs
    #img_arr_std = np.clip(img_arr_std, -1.0, 1.0)
    
    # shift to 0--1
    #img_arr_std = (img_arr_std + 1.0) / 2.0
    
    return img_arr_std

In [12]:
def normalize_pixels(img_arr: np.array) -> np.array:
    img_arr_norm = img_arr / 255.0
    return img_arr_norm.astype("float16")    

In [13]:
def image_cripple(img_arr: np.array, max_nan_pct: float) -> np.array:
    #Ajoute des NaN à une image pour en dégrader la qualité
    # Si un pixel est NaN il l'est dans toutes les couleurs
    assert len(img_arr.shape) == 3 #3D
    assert img_arr.shape[2] == 3 #Image en 3 couleurs
    assert (0.0 <= max_nan_pct <= 1.0)
    
    import random
    result = np.array(img_arr)
    
    pixel_count = img_arr.shape[0] * img_arr.shape[1]
    pixel_nan_count = int(pixel_count * max_nan_pct)
    
    #Liste de coordonnées aléatoires
    pixel_nan_list = [(random.randrange(0, img_arr.shape[0]), random.randrange(0, img_arr.shape[1])) 
                      for i in range(pixel_nan_count)]
    
    for pixel_nan in pixel_nan_list:
        result[pixel_nan[0], pixel_nan[1]]=np.nan
        
    return result

L'algorithme de patchage est imparfait. 

Il prend les plus proches voisins non Nan dans 8 directions.

Une meilleur méthode consisterai à trouver le contours du trou et ensuite de faire la moyenne des pixels sur le périmètre.

In [14]:
def image_patch(img_arr: np.array) -> np.array:
    #Remplace les NaN par la valeur moyenne des pixels alentours
    # Si un pixel est NaN il l'est dans toutes les couleurs
    assert len(img_arr.shape) == 3 #3D
    assert img_arr.shape[2] == 3 #Image en 3 couleurs

    result=np.array(img_arr)
    #=== Trouve les pixels à Nan
    # Donne les lignes dans une liste et les colonnes dans une autre liste
    pixel_nan_row_column_list = np.where(np.isnan(img_arr[:,:,0]))
    # Crée des coordonnées (Ligne, colonne) à partir des listes précédentes.
    pixel_nan_list = [(row, column)
                      for row, column in zip(pixel_nan_row_column_list[0], 
                                            pixel_nan_row_column_list[1])]
    #--- Trouve les pixels à Nan
    
    #=== Patch les pixels à Nan
    for pixel_nan in pixel_nan_list:
        pixel_neighbourhood=find_neighbourhood(pixel_nan, img_arr)
        pixel_patched = patch_pixel(pixel_neighbourhood)
        result[pixel_nan[0], pixel_nan[1]]=pixel_patched
    #---Patch les pixels à Nan
    
    return result
        

In [15]:
def find_neighbourhood(pixel_nan: tuple, img_arr: np.array) -> np.array:
    #Trouve les pixels avoisinants non Nan
    # On cherche horizontalement, verticalement et en diagonale et on prend le 1er qu'on rencontre.
    assert len(pixel_nan)==2                                  #Vérifie si ce sont bien des coordonnées (row, column)
    assert np.isnan(img_arr[pixel_nan[0], pixel_nan[1]]).sum()>0 #Vérifie que c'est bien un pixel Nan
    assert len(img_arr.shape) == 3                            #3D
    assert img_arr.shape[2] == 3                              #Image en 3 couleurs
    
    pixel_neighbourhood=np.empty((3, 3, 3), dtype = type(img_arr[0,0,0]))
    pixel_neighbourhood[:]=np.nan

    for row_direction in [-1, 0, 1]:

        for column_direction in [-1, 0, 1]:

            if (row_direction !=0 and column_direction !=0):

                #Trouve le 1er pixel non NaN dans la direction considérée
                for i in range(max(img_arr.shape[0], img_arr.shape[1])):
                    row = pixel_nan[0]
                    column = pixel_nan[1]

                    row = row + (i * row_direction)
                    column = column + (i * column_direction)

                    if 0 <= row < img_arr.shape[0] and 0 <= column < img_arr.shape[1]:
                        if not np.isnan(img_arr[row, column, 0]):
                            pixel_neighbourhood[row_direction + 1, column_direction + 1]=img_arr[row, column]
                            break #Sort de la boucle de recherche

    return pixel_neighbourhood

In [16]:
def patch_pixel(pixel_neighbourhood: np.array) -> np.array:
    #Calcule la moyenne des pixels avoisinants
    #Le pixel résultat est au centre du voisinage
    assert len(pixel_neighbourhood.shape) == 3 #3D
    assert pixel_neighbourhood.shape[2] == 3   #3 couleurs
    result = np.zeros(3, dtype = "float16")
    
    for i in range(len(result)):
        result[i]=np.nanmean(pixel_neighbourhood[:,:,i])

    return result    

In [17]:
# NBR_SAMPLE = 1000

In [18]:
df_path_img_healthy_sample = df_path_img_healthy.sample(n = int(NBR_SAMPLE/2), random_state=0)
df_path_img_sick_sample    = df_path_img_sick.sample(n = int(NBR_SAMPLE/2), random_state=0)

In [19]:
PATH_DATA_SAMPLE

'../data/sample'

In [20]:
def preprocess_img(path_img:str, img_size:int=IMG_SIZE) -> np.array:
        
    #img_arr_std = positive_global_std(image_resize(path_img))
    img_arr_preprocessed = normalize_pixels(image_resize(path_img))
    # padding
    img_arr_padded = np.zeros((IMG_SIZE, IMG_SIZE, 3), dtype = type(img_arr_preprocessed[0,0,0]))
    img_arr_padded[:img_arr_preprocessed.shape[0],:img_arr_preprocessed.shape[1],:] = img_arr_preprocessed
    
    return img_arr_padded

In [21]:
# Export des fonctions pour le pré-traitement des images
!pip install dill
import dill
preprocessing_function_list = [normalize_pixels, preprocess_img]

for preprocessing_function in preprocessing_function_list:
    print("Export de [" + preprocessing_function.__name__ + "]")
    with open(preprocessing_function.__name__ + ".dill", "wb") as file:
        dill.dump(preprocessing_function, file)

Export de [normalize_pixels]
Export de [preprocess_img]


In [22]:
type(normalize_pixels)

function

In [23]:
DATE_TIME = "2021-11-18_20-39-00"

def create_sample(df_path_img:str, img_type:str, path_sample:str, date_time:str, clean_dir:bool):
    path_sample_time = path_sample+"/"+date_time

    os.makedirs(path_sample_time, exist_ok=True)
    if clean_dir:
        file_list = glob.glob(path_sample_time+"/"+"*.*")
        for f in file_list:
            os.remove(f)
    
    for path_img , _ in zip(df_path_img.path_img, tqdm(range(len(df_path_img.path_img)))):
        
        _, img_name, _ = utils.split_file_info(path_img)
        
        img_arr_padded = preprocess_img(path_img)
        
        utils.pickle_save(img_arr_padded, path_sample_time + "/" + str(img_type).upper() + "_" + img_name + ".pklz")
        

In [24]:
create_sample(df_path_img_healthy_sample, HEALTHY, PATH_DATA_SAMPLE, DATE_TIME, clean_dir=True)

100%|████████████████████████████████████████████████████████████████████████████▉| 9999/10000 [03:26<00:00, 48.42it/s]


In [25]:
create_sample(df_path_img_sick_sample, SICK, PATH_DATA_SAMPLE, DATE_TIME, clean_dir=False)

100%|████████████████████████████████████████████████████████████████████████████▉| 9999/10000 [03:37<00:00, 46.07it/s]


In [26]:
df_path_img_sample = pd.concat([df_path_img_healthy_sample, df_path_img_sick_sample], ignore_index=True)
df_path_img_not_in_sample = pd.DataFrame([i for i in df_path_img.path_img.values if not i in df_path_img_sample.path_img.values], 
                 columns=df_path_img_sample.columns)

In [27]:
df_path_img_not_in_sample.shape

(7558, 1)

In [28]:
df_path_img_not_in_sample.head()

,path_img
0,../cell_images/Parasitized\C100P61ThinF_IMG_20...
1,../cell_images/Parasitized\C100P61ThinF_IMG_20...
2,../cell_images/Parasitized\C100P61ThinF_IMG_20...
3,../cell_images/Parasitized\C100P61ThinF_IMG_20...
4,../cell_images/Parasitized\C100P61ThinF_IMG_20...


In [29]:
df_path_img_not_in_sample.sample(1).values

array([['../cell_images/Uninfected\\C62P23N_ThinF_IMG_20150818_132811_cell_9.png']],
      dtype=object)